<img width="8%" alt="Google Maps.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Maps.png" style="border-radius: 15%">

# Google Maps - Create and display distance matrix
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Google+Maps+-+Create+and+display+distance+matrix:+Error+short+description">Bug report</a>

**Tags:** #googlemaps #productivity #operations #automation #jupyternotebooks

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

**Last update:** 2023-08-08 (Created: 2023-08-08)

**Description:** This notebook shows how to use the Google Maps Distance Matrix API to determine distances and trip durations between a number of different origins and destinations, giving accurate and efficient geospatial data. Furthermore, it provides a visual representation of the created matrix.

## Input

### Import libraries

In [ ]:
import os
import requests
import naas
import folium
import googlemaps
import polyline
from geopy.geocoders import Nominatim

### Setup variables
**Mandatory**

Pre-requisite: 
1. Follow the steps in the link - https://developers.google.com/maps/documentation/routes/cloud-setup
2. Sign up for an account with the Routes API provider.
3. Make sure to enable Routes API as it is in the "Additional APIs" section in the google cloud console.
4. Check the available countries https://developers.google.com/maps/documentation/routes/calculate_toll_fees#toll-locations

- `api_key`: This variable holds the Google Cloud Platform (GCP) API key.
- `origins`: This variable represents the starting locations for a journey.
- `destinations`: This variable indicates the end location or destination for a journey.

**Optional**
- `colors`: List of colors to be used to diplay routes

In [ ]:
# Mandatory
api_key = naas.secret.get("GCP_MAP_API_KEY") or "YOUR_GCP_MAP_API_KEY"  # Read API key from the secret variable GCP_MAP_API_KEY
origins = ["New York, NY"]
destinations = ["San Francisco, CA", "Miami, FL", "Seattle, WA"]

# Optional
colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow']

## Model

### Connect to Google Maps

In [ ]:
gmaps = googlemaps.Client(key=api_key)

### Get distance matrix

In [ ]:
def get_distance_matrix(api_key, origins, destinations, mode="driving", units="metric"):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "key": api_key,
        "origins": "|".join(origins),
        "destinations": "|".join(destinations),
        "mode": mode,
        "units": units
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] == "OK":
        return data["rows"]
    else:
        print(f"Error: {data['status']}")
        return None
    
matrix = get_distance_matrix(api_key, origins, destinations)

## Output

### Process the data retrieved by the distance matrix 

In [ ]:
if matrix:
    map_routes = folium.Map(location=[37.7749, -122.4194], zoom_start=4)

    for origin in origins:
        location = Nominatim(user_agent="geoapiExercises").geocode(origin)
        folium.Marker([location.latitude, location.longitude], popup=folium.Popup(location.address)).add_to(map_routes)

    for destination in destinations:
        location = Nominatim(user_agent="geoapiExercises").geocode(destination)
        folium.Marker([location.latitude, location.longitude], popup=folium.Popup(location.address)).add_to(map_routes)

    for origin_index, origin in enumerate(matrix):
        for destination_index, destination in enumerate(origin["elements"]):
            if "status" in destination and destination["status"] == "OK":
                origin_location = origins[origin_index]
                destination_location = destinations[destination_index]
                directions = gmaps.directions(origin_location, destination_location, mode="driving")
                if directions:
                    color_index = (origin_index + destination_index) % len(colors)  # Fixed color_index calculation
                    for step in directions[0]['legs'][0]['steps']:
                        polyline_data = step['polyline']['points']
                        decoded_points = polyline.decode(polyline_data)
                        folium.PolyLine(locations=decoded_points, color=colors[color_index]).add_to(map_routes)

for origin_index, origin in enumerate(matrix):
    for destination_index, destination in enumerate(origin["elements"]):
        distance = destination["distance"]["text"]
        duration = destination["duration"]["text"]
        color_index = (origin_index + destination_index) % len(colors)
        print(f"🗺️ From {origins[origin_index]} to {destinations[destination_index]}")
        print(f"-Route color: {colors[color_index]} \n-Distance: {distance} \n-Duration: {duration}")
        print()
        
map_routes